In [ ]:
import pandas as pd
import numpy as np
import json
import random
import nltk
#nltk.download('stopwords')
#stop = stopwords.words('english')

In [ ]:
#Reading the partial file that was created
data = pd.read_csv('new_data.csv')
data.head()

,Unnamed: 0,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,0,math/9303206,NaN,Mikhail I. Ostrovskii,Total subspaces with long chains of nowhere no...,NaN,"Note Mat., 13 (1993), no. 2, 217-227",NaN,Banach Archive 3/29/93,math.FA,NaN,If a separable Banach space $X$ is such that...,"[{'version': 'v1', 'created': 'Mon, 29 Mar 199...",2010-09-07,"[['Ostrovskii', 'Mikhail I.', '']]"
1,1,1906.01159,Lucas Daguerre,Lucas Daguerre,Estudio del crecimiento del Sn/Ag(111): explor...,"12 Pages, in Spanish, 17 Figures",NaN,NaN,NaN,cond-mat.mes-hall,http://arxiv.org/licenses/nonexclusive-distrib...,"In recent years, 2D materials have attracted...","[{'version': 'v1', 'created': 'Tue, 4 Jun 2019...",2019-06-05,"[['Daguerre', 'Lucas', '']]"
2,2,1208.1559,Keiko Kawamuro,Tetsuya Ito and Keiko Kawamuro,Essential open book foliation and fractional D...,"58 pages, 35 figures",NaN,NaN,NaN,math.GT math.SG,http://arxiv.org/licenses/nonexclusive-distrib...,We introduce an essential open book foliatio...,"[{'version': 'v1', 'created': 'Wed, 8 Aug 2012...",2015-09-02,"[['Ito', 'Tetsuya', ''], ['Kawamuro', 'Keiko',..."
3,3,1209.0623,Imme van den Berg,Imme van den Berg,Stochastic differential equations with path-in...,NaN,NaN,NaN,NaN,math.PR,http://arxiv.org/licenses/nonexclusive-distrib...,We present a condition for a stochastic diff...,"[{'version': 'v1', 'created': 'Tue, 4 Sep 2012...",2012-09-05,"[['Berg', 'Imme van den', '']]"
4,4,1806.02676,"Christian B\""ohning","Asher Auel, Alessandro Bigazzi, Christian B\""o...",Universal triviality of the Chow group of 0-cy...,12 pages; v2: improvements to exposition and n...,NaN,NaN,NaN,math.AG,http://arxiv.org/licenses/nonexclusive-distrib...,We prove that a smooth proper universally CH...,"[{'version': 'v1', 'created': 'Thu, 7 Jun 2018...",2018-06-19,"[['Auel', 'Asher', ''], ['Bigazzi', 'Alessandr..."


In [ ]:
keywords = pd.read_csv('Keywords.csv', header=None)

#removing that one null row
keywords = keywords[keywords[0].isnull() == False]

#dropping all the null columns
keywords = keywords.dropna(axis=1)

#resets indices to fill up the missing value from the previous step
keywords = keywords.reset_index(drop=True)

#dropping rows that contain stop words since they will not provide us with anything useful
# keywords["temp"] = [True  if row[0] not in stop else False for index, row in keywords.iterrows()]
# keywords = keywords[keywords.temp == True]
# keywords.drop('temp', axis=1, inplace=True)

keywords = keywords[:20000]

print(keywords)

                                0     1
0                machine learning  5933
1               genetic algorithm  5264
2                  classification  4532
3                   deep learning  4371
4                     data mining  4369
...                           ...   ...
19995  genetic algorithm approach    21
19996               learning tool    21
19997                data channel    21
19998      quality characteristic    21
19999              schema version    21

[20000 rows x 2 columns]


In [ ]:
#Converting to list because pandas is trash in terms of efficiency

keyword_list = keywords[0].values.tolist()
abs_list = data['abstract'].values.tolist()

print(keyword_list)
print(len(abs_list))

['machine learning', 'genetic algorithm', 'classification', 'deep learning', 'data mining', 'cloud computing', 'support vector machine', 'clustering', 'model check', 'security', 'optimization', 'feature selection', 'neural network', 'multiagent system', 'simulation', 'neural networks', 'artificial intelligence', 'ontology', 'wireless sensor network', 'logic program', 'virtual reality', 'particle swarm optimization', 'privacy', 'scheduling', 'face recognition', 'logic programming', 'augmented reality', 'operational semantic', 'big data', 'information retrieval', 'natural language processing', 'feature extraction', 'wireless sensor networks', 'independent component analysis', 'mobile robot', 'business process', 'fuzzy logic', 'image segmentation', 'image processing', 'internet of things', 'segmentation', 'convolutional neural network', 'temporal logic', 'hash function', 'usability', 'computer vision', 'reinforcement learning', 'evaluation', 'computational mathematic', 'cellular automaton

In [ ]:
#TIME TAKING CELL - 1 and a half minute for 500 iterations (250,000 abstracts) - total 1 hr 20 minutes
#replaces spaces with underscores for the key words present in each abstract AND IN EACH KEYWORD
#Need to test with the full set of keywords
cnt=0
for i in range(len(keyword_list)):
  word = str(keyword_list[i])
  replacement = word.replace(' ', '_')
  keyword_list[i] = replacement

  abs_list = [sub.replace(word, replacement) for sub in abs_list]
  cnt+=1
  if cnt%500==0:
    print(cnt)

#print(abs_list[:100])

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000


In [ ]:
print(abs_list[3054])
print('hamiltonian' in keyword_list)

  We consider the point_process of signal_strengths from transmitters in a
wireless_network observed from a fixed position under models with general
signal path_loss and random propagation effects. We show via coupling arguments
that under general conditions this point_process of signal_strengths can be
well-approximated by an inhomogeneous Poisson or a Cox point_processes on the
positive real_line. We also provide some bounds on the total_variation distance
between the laws of these point_processes and both Poisson and Cox point
processes. Under appropriate conditions, these results support the use of a
spatial Poisson point_process for the underlying positioning of transmitters in
models of wireless_networks, even if in reality the positioning does not appear
Poisson. We apply the results to a number of models with popular choices for
positioning of transmitters, path_loss_functions, and distributions of
propagation effects.

True


In [ ]:
#TIME TAKING CELL - 2 hours and 45 minutes
#Removing all words that are not in the list of keywords
#Didn't see the exact time but it was done by the time I ate dinner

#temp_data = temp_data.assign(abstract = temp_data['abstract'].apply(lambda x: ' '.join([word for word in x.split() if word in (keyword_list)])))
#temp_data = temp_data.assign(abstract = temp_data['abstract'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)])))
#data = data.assign(abstract = data['abstract'].apply(lambda x: ' '.join([word for word in x.split() if word in (keyword_list)])))

abs_list = [' '.join([word for word in abs.split() if word in (keyword_list)]) for abs in abs_list]

In [ ]:
#remove all the duplicates from each abstract to reduce the number of further comparisons even more
def unique_list(l):
    ulist = []
    [ulist.append(x) for x in l if x not in ulist]
    return ulist

cnt=0
for i in range(len(abs_list)):
  abs_list[i] = ' '.join(unique_list(abs_list[i].split()))
  cnt+=1
print(cnt)

250000


In [ ]:
#Putting both the abstracts and keywords into csv files
new_abstracts = pd.DataFrame(abs_list)
new_abstracts.to_csv('modified_abstracts.csv', index=False)

In [ ]:
new_keywords = pd.DataFrame(keyword_list)
new_keywords.to_csv('modified_keywords.csv', index=False)
print(abs_list[19000])

human time history disaster it quantum interpretation


In [ ]:
#Creating an adjacency list with key words as nodes

adj = {}
for word in keyword_list:
  adj[word] = {}

cnt=0
for abs in range(len(abs_list)):
  words = abs_list[abs].split()
  #print(words[0])
  
  for i in range(len(words)):
    for j in range(i+1, len(words)):
      try:
        adj[words[i]][words[j]]+=1
      except KeyError:
        adj[words[i]][words[j]] = 1
      try:
        adj[words[j]][words[i]]+=1
      except KeyError:
        adj[words[j]][words[i]] = 1

  cnt+=1
  if cnt%1000==0:
    print(cnt)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [ ]:
#writing the dictionary of dictionaries into a json file so I don't die with running this whole thing later
with open('adjacency_list.json', 'w') as fp:
    json.dump(adj, fp)

In [ ]:
print(keyword_list[:15])

['machine_learning', 'genetic_algorithm', 'classification', 'deep_learning', 'data_mining', 'cloud_computing', 'support_vector_machine', 'clustering', 'model_check', 'security', 'optimization', 'feature_selection', 'neural_network', 'multiagent_system', 'simulation']


In [ ]:
print(adj['machine_learning']['linear_regression'])

6


In [ ]:
print(adj['data_mining'])

NameError: ignored